## PHASE I

In [ ]:
pip install hazm

In [5]:
from __future__ import unicode_literals
import pandas as pd
import numpy as np
from hazm import *
import collections 
import string
import operator
import datetime
import json
import math
import copy
import re

In [ ]:
# upload the file 
from google.colab import files

uploaded = files.upload()


In [5]:
# Read file from the Google colud
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1jcbbbPTNnQ3NKrPJJL_oc9vrFvHH4UuR'
downloaded = drive.CreateFile({'id': file_id})

In [ ]:
downloaded.GetContentFile('/IR1_7k_news.xlsx')

In [8]:
# read data
# df = pd.read_excel("./IR1_7k_news.xlsx")
df = pd.read_excel("/IR1_7k_news.xlsx")
urls = df['url']
content = df['content']
title = df['title']
# normalize data by use hazm
normalizer = Normalizer()
print(content[11])
for i in range(len(content)):
  content[i] = normalizer.normalize(content[i])
copy_content = content
print(content[11])

به گزارش خبرگزاری فارس از **اصفهان،** دیدار تیم‌های ذوب‌آهن و استقلال با 2 گل
به سود شاگردان [فرهاد مجیدی](https://www.farsnews.ir/special/فرهاد مجیدی) به
پایان رسید.

بعد از این مسابقه [رشید مظاهری](https://search.farsnews.ir/?q=رشید
مظاهری&o=on) در رختکن ذوب‌آهن حاضر شد و با بازیکنان تیم میزبان خوش وبش کرد.
مظاهری بعد از خروج از رختکن به درخواست خبرنگاران برای مصاحبه پاسخ منفی داد و
گفت آنقدر مشکل روحی و روانی دارم که اصلاً نمی‌توانم مصاحبه کنم.

به گزارش فارس، ظاهراً طی روزهای گذشته مشکلاتی برای دروازه‌بان استقلال در
تمرینات این تیم به وجود آمده که وی [به صورت](https://search.farsnews.ir/?q=به
صورت&o=on) سربسته مقابل خبرنگاران به این موضوع اشاره کرده است.

انتهای پیام/


به گزارش خبرگزاری فارس از **اصفهان، ** دیدار تیم‌های ذوب‌آهن و استقلال با ۲ گل
به سود شاگردان [فرهاد مجیدی] (https: //www. farsnews. ir/special/فرهاد مجیدی) به
پایان رسید. 

بعد از این مسابقه [رشید مظاهری] (https: //search. farsnews. ir/?q=رشید
مظاهری&o=on) در رختکن ذوب‌آهن حاضر شد و با بازیکنان تیم میزبان خوش وبش ک

In [9]:
punctuation  = [ '+','=', '-' , '*' , '.' ,'?', '[', ']','(',')','{','}','<','>', '«' , '»' ,':' , "؟", "؛" , "،", "،" ]
signs = [ '!' , '@' , '#' , '$' , '%' , '^' , '&' , '*', '_', '\\' , '/' , '//' , '|' , "…" , "–" ,"_" , "   "]
numbers = ['۰' , '۱', '۲','۳', '۴', '۵','۶','۷','۸','۹']
signs_ir = ['٪' , ',' , '_','–' , '٫' , '"' ]
img_src = ['UFITNPF']
end = ['انتهای پیام']
# some_unknown_char = ['\u200c' , '\u200d' , '\u200e' , '\u200f']
english = list(string.ascii_lowercase) + list(string.ascii_uppercase)
not_used = punctuation + signs + numbers + signs_ir + img_src + english + end 
print(not_used)

['+', '=', '-', '*', '.', '?', '[', ']', '(', ')', '{', '}', '<', '>', '«', '»', ':', '؟', '؛', '،', '،', '!', '@', '#', '$', '%', '^', '&', '*', '_', '\\', '/', '//', '|', '…', '–', '_', '   ', '۰', '۱', '۲', '۳', '۴', '۵', '۶', '۷', '۸', '۹', '٪', ',', '_', '–', '٫', '"', 'UFITNPF', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'انتهای پیام']


In [10]:
stopwords = set(stopwords_list())
print(len(stopwords))

389


اینجا در ابتدا میبایستی ما یکسری از حروف‌های بلا استفاده رو حذف کنیم

In [11]:
# Make the change for the not_used word for us
for i in range(len(content)):
    for l in not_used:
        content[i] = content[i].replace(l,"")


In [ ]:
# make copy from the content that preprocess them (it used ^_^)
preprocess_content =copy.deepcopy(content)

In [ ]:
print(content[11])
print(content[9])

In [ ]:
# make the stem of the word
lstemmer = Stemmer()
lstemmer.stem('مظاهری')

'مظاهر'

In [ ]:
# the verb stem
lemmatizer = Lemmatizer()
verbs = lemmatizer.lemmatize('مظاهری')
x = verbs.split('#')
print(x)

['مظاهر']


In [11]:
# get tokenize by using the hazm word tokenize
normalizer = Normalizer()
dictionary_non_posting = collections.defaultdict(list)
for i in range(len(content)):
  tokenizes = []
  tokenizes = word_tokenize(content[i])
  tokenizes_count = dict(collections.Counter(tokenizes))
  for key , value in tokenizes_count.items():
    if key in dictionary_non_posting:
      value_old = dictionary_non_posting[key]
      dictionary_non_posting[key] = value + value_old
    else:
      dictionary_non_posting[key] = value 

In [12]:
print((dictionary_non_posting['و']))
print(len(dictionary_non_posting))

130255
58734


In [13]:
length_dict_non_posting = {k: v for k, v in sorted(dictionary_non_posting.items(), key=lambda item: item[1] , reverse=True)}
with open("mydict.txt", 'w') as f: 
    for key, value in length_dict_non_posting.items(): 
        f.write("%s:%s\n" % (key, value))

In [14]:
stop_words = ['آنها' , 'پیش' ,'پس' ,'هر' ,'او' ,'یا' ,'نیز' ,'وی' ,'ما' ,'خود' ,'هم' ,'تا' ,'آن' ,'بر' ,'برای' ,'را' ,'با' ,'که' ,'این' ,'از' ,'به' ,'در' ,'و' ]
stopwords.update(stop_words)
print(len(stopwords))

389


In [16]:
# to find position of each word in contents
def position_find(word_to_find , lists):
  # words = contents.split()
  return  [pos for pos, word in enumerate(lists, start=0) if ((word == word_to_find) )] 


In [16]:
# get tokenize by using the hazm word tokenize (the postings list [docid , ferq , [position]])
normalizer = Normalizer()
dictionary_not_steam = collections.defaultdict(list)
for i in range(len(content)):
    tokenizes = []
    tokenizes = word_tokenize(content[i])
    tokenizes_count = dict(collections.Counter(tokenizes))
    for key , value in tokenizes_count.items():
      if((key not in stopwords)):
        index = []
        pos_index = position_find(key , tokenizes)
        index.append(i)
        index.append(value)
        index.append(pos_index)
        dictionary_not_steam[key].append(index)
    

In [ ]:
print(dictionary_not_steam['و'])
print(len(dictionary_not_steam))
print(dictionary_not_steam['یحیی'])
print(len(dictionary_not_steam['یحیی']))

[]
58347
[[1, 1, [126]], [47, 2, [160, 163]], [49, 1, [374]], [84, 2, [60, 63]], [128, 1, [50]], [132, 2, [55, 58]], [177, 2, [482, 485]], [223, 1, [6]], [226, 2, [19, 22]], [227, 1, [248]], [231, 1, [168]], [237, 1, [4]], [251, 2, [6, 9]], [268, 2, [4, 7]], [485, 2, [23, 100]], [532, 2, [246, 249]], [601, 1, [50]], [647, 2, [6, 9]], [652, 1, [15]], [653, 1, [183]], [654, 1, [163]], [656, 1, [89]], [659, 1, [104]], [665, 2, [28, 31]], [733, 1, [90]], [736, 1, [6]], [768, 1, [75]], [786, 2, [6, 9]], [791, 2, [83, 86]], [809, 2, [314, 317]], [814, 2, [43, 46]], [816, 1, [139]], [828, 1, [134]], [863, 2, [199, 202]], [864, 2, [78, 81]], [880, 1, [142]], [894, 2, [62, 240]], [928, 1, [14]], [932, 1, [57]], [964, 2, [337, 340]], [986, 1, [261]], [995, 2, [15, 18]], [1013, 2, [16, 83]], [1018, 1, [48]], [1043, 1, [361]], [1087, 1, [11]], [1105, 1, [6]], [1124, 3, [430, 815, 818]], [1133, 2, [16, 19]], [1157, 1, [30]], [1165, 1, [11]], [1177, 1, [35]], [1183, 1, [541]], [1191, 1, [137]], [121

In [14]:
lstemmer = Stemmer()
lemmatizer = Lemmatizer()
def stem_stop(words_content):
  for j in range(len(words_content)):
        words_content[j] = lstemmer.stem(words_content[j])
  return words_content

In [18]:
# get tokenize by using the hazm word tokenize (the postings list [docid , ferq , [position]])
dictionary = collections.defaultdict(list)
for i in range(len(content)):
    tokenizes = []
    tokenizes = word_tokenize(content[i])
    tokenizes = stem_stop(tokenizes)
    tokenizes_count = dict(collections.Counter(tokenizes))
    for key , value in tokenizes_count.items():
        if key not in stopwords:
          index = []
          pos_index = position_find(key , tokenizes)
          index.append(i)
          index.append(value)
          index.append(pos_index)
          dictionary[key].append(index)

In [ ]:
print(len(dictionary))

39436


In [19]:
# write the postings list in json 
with open("posting-list.json", "w") as outfile:
    json.dump(dictionary, outfile)


In [ ]:
# Read postings list from json
with open('posting-list.json') as json_file:
    dictionary = json.load(json_file)

In [ ]:
print(len(dictionary[lstemmer.stem('و')]))
print((dictionary[lstemmer.stem('')]))
print(dictionary_not_steam[('بین‌الملل')])
print(len(dictionary_not_steam[('بین‌الملل')]))


0
[[0, 1, [138]], [2, 6, [528, 590, 661, 1098, 1149, 1336]], [3, 2, [68, 198]], [5, 15, [31, 86, 109, 148, 926, 1016, 1041, 1537, 2088, 2168, 2248, 2465, 3388, 4121, 4558]], [9, 1, [90]], [12, 3, [59, 209, 327]], [15, 1, [109]], [23, 1, [240]], [26, 1, [36]], [28, 2, [23, 29]], [29, 1, [184]], [31, 2, [60, 182]], [35, 1, [62]], [39, 1, [108]], [46, 1, [28]], [49, 1, [415]], [52, 1, [37]], [67, 1, [158]], [68, 2, [361, 434]], [69, 1, [45]], [74, 1, [161]], [77, 1, [190]], [78, 1, [211]], [81, 1, [194]], [85, 2, [98, 119]], [92, 1, [137]], [97, 3, [222, 383, 404]], [98, 1, [53]], [121, 1, [53]], [126, 4, [62, 171, 482, 531]], [127, 1, [165]], [132, 2, [497, 541]], [133, 3, [14, 141, 154]], [138, 2, [39, 511]], [141, 2, [41, 296]], [143, 1, [42]], [144, 1, [147]], [146, 1, [465]], [153, 1, [297]], [154, 2, [14, 41]], [158, 1, [18]], [162, 2, [11, 65]], [176, 1, [547]], [177, 2, [125, 415]], [182, 1, [86]], [183, 1, [186]], [187, 2, [36, 37]], [189, 2, [104, 793]], [190, 1, [300]], [204, 3

In [ ]:
print(len(dictionary[lstemmer.stem('صنعتی')]))
print((dictionary[lstemmer.stem('صنعتی')]))
print(dictionary_not_steam[('بین‌الملل')])
print(len(dictionary_not_steam[('بین‌الملل')]))


235
[[311, 1, [351]], [362, 1, [196]], [480, 2, [38, 130]], [521, 1, [202]], [535, 1, [534]], [566, 1, [159]], [798, 1, [434]], [1316, 1, [149]], [1362, 1, [8]], [1368, 1, [540]], [1674, 2, [61, 121]], [1725, 1, [79]], [1753, 4, [1606, 4044, 4205, 4327]], [1773, 3, [209, 432, 1302]], [1779, 1, [382]], [1784, 1, [47]], [1805, 1, [557]], [1825, 1, [294]], [1827, 1, [490]], [1830, 1, [751]], [1863, 2, [199, 426]], [1870, 2, [34, 209]], [1901, 1, [850]], [1916, 1, [1004]], [1932, 1, [791]], [1937, 1, [133]], [1952, 5, [226, 295, 330, 631, 655]], [1959, 1, [570]], [1977, 1, [54]], [1980, 1, [321]], [1991, 1, [570]], [2039, 1, [148]], [2045, 1, [2114]], [2063, 2, [346, 382]], [2066, 2, [386, 2460]], [2067, 1, [357]], [2079, 1, [73]], [2089, 1, [1720]], [2091, 2, [29, 211]], [2092, 2, [48, 117]], [2100, 1, [31]], [2126, 1, [678]], [2131, 1, [500]], [2153, 1, [830]], [2163, 1, [139]], [2198, 8, [4102, 4106, 4125, 4134, 4138, 4142, 4146, 4151]], [2217, 1, [1624]], [2259, 1, [71]], [2302, 1, [88

In [20]:
# the more priority
def sort_dict(dictionary):
   dicts = dictionary
   sorted_dicts = sorted(dicts, key = operator.itemgetter(0, 1) , reverse=False)
   all = 0
   for i in range(len(sorted_dicts)):
     all = all + sorted_dicts[i][1]
   return all , sorted_dicts

In [31]:
# Enter With wrong keyboard language
def _multiple_replace(mapping, text):
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

def convert_en_characters(input_str):
    mapping = {
        'q': 'ض',
        'w': 'ص',
        'e': 'ث',
        'r': 'ق',
        't': 'ف',
        'y': 'غ',
        'u': 'ع',
        'i': 'ه',
        'o': 'خ',
        'p': 'ح',
        '[': 'ج',
        ']': 'چ',
        'a': 'ش',
        's': 'س',
        'd': 'ی',
        'f': 'ب',
        'g': 'ل',
        'h': 'ا',
        'j': 'ت',
        'k': 'ن',
        'l': 'م',
        ';': 'ک',
        "'": 'گ',
        'z': 'ظ',
        'x': 'ط',
        'c': 'ز',
        'v': 'ر',
        'b': 'ذ',
        'n': 'د',
        'm': 'پ',
        ',': 'و',
        '?': '؟',
        'C': 'ژ',
        'H': 'آ',
    }
    return _multiple_replace(mapping, input_str)

In [22]:
convert_en_characters("Cdlkhsjd;")

'ژیمناستیک'

In [23]:
# one word query 
# this sorted query question by frequency
def query_one_word():
  query = input("enter a word for checking: ")
  query = convert_en_characters(query)
  time_start = datetime.datetime.now()
  normal_query = lstemmer.stem(query)
  print("Normal word to search {}".format(normal_query))
  # all ,sorted_dict = sort_dict(dictionary[normal_query])
  dicts = dictionary[normal_query]
  sorted_dict = sorted(dicts, key = operator.itemgetter(1, 2) , reverse=True)
  print(sorted_dict)
  time_finish = datetime.datetime.now()
  print("{} results in {} ms".format(len(sorted_dict), ((time_finish - time_start).total_seconds())*1000))
  print("id -> title\n")
  for i in sorted_dict:
    print("{} -> {}".format(i[0] , title[i[0]]))
  print(sorted_dict)



In [24]:
query_one_word()

enter a word for checking: Cdlkhsjd;
Normal word to search ژیمناستیک
[[1367, 12, [15, 48, 126, 132, 145, 185, 231, 245, 281, 287, 300, 306]], [632, 12, [14, 35, 70, 137, 154, 164, 223, 237, 261, 324, 339, 380]], [4188, 3, [210, 409, 623]], [3664, 3, [10, 65, 114]], [3878, 2, [227, 424]], [1456, 2, [33, 254]], [4056, 1, [718]], [3615, 1, [10]]]
8 results in 0.187 ms
id -> title

1367 -> هشدار هیات ژیمناستیک تهران در خصوص سالن‌های مختلط و اقدامات غیراخلاقی
632 -> خیرخواه: برخی به دنبال فلج کردن ژیمناستیک هستند/ با بایکوت فدراسیون موفقیت‌ها بیشتر شد
4188 -> جزییات تعطیلی‌های ورزش ایران تا ۹ مهر ۱۴۰۰/ کدام فعالیت‌های ورزشی در تهران ممنوع است؟
3664 -> ثبت نام ۱۳ نامزد برای پست ریاست فدراسیون ژیمناستیک + اسامی
3878 -> جزییات تعطیلی ورزش ایران تا پایان تیرماه+ تصویر
1456 -> سوت‌زنی| تذکر و برخورد با سالن‌های ورزشی مختلط توسط وزارت ورزش
4056 -> دبیر: اگر من در مباحث فنی ۱۰ باشم، درستکار ۱۰۰ است/ بنا کاملا بر اساس چرخه انتخابی عمل کرد!
3615 -> دبیر مجمع فدراسیون ژیمناستیک مشخص شد
[[1367, 12, [1

In [25]:
# the dictionary values is the posting lists
# post_list = dictionary[key][i] 
# the i is iteated 
# return docID
def docID(post_list):
        return post_list[0]

In [26]:
# the dictionary values is the posting lists
# post_list = dictionary[key][i] 
# the i is iteated 
# return list of position
def position(plist):
        return plist[2]    

In [27]:
# position intersection algorithm
# get the to word position list and start find k position for p1 , p2
def position_intersect(p1,p2,k):
        answer = []                                                                     # answer <- ()
        len1 = len(p1)
        len2 = len(p2)
        i = j = 0 
        while i != len1 and j != len2:                                                  # while (p1 != nil and p2 != nil)
                if docID(p1[i]) == docID(p2[j]):
                        l = []                                                          # l <- ()
                        pp1 = position(p1[i])                                           # pp1 <- positions(p1)
                        pp2 = position(p2[j])                                           # pp2 <- positions(p2)
    
                        plen1 = len(pp1)
                        plen2 = len(pp2)
                        ii = jj = 0 
                        while ii != plen1:                                              # while (pp1 != nil)
                                while jj != plen2:                                      # while (pp2 != nil)
                                        if abs(pp1[ii] - pp2[jj]) <= k:                 # if (|pos(pp1) - pos(pp2)| <= k)
                                                l.append(pp2[jj])                       # l.add(pos(pp2))
                                        elif pp2[jj] > pp1[ii]:                         # else if (pos(pp2) > pos(pp1))
                                                break    
                                        jj+=1                                           # pp2 <- next(pp2)      
                                while l != [] and abs(l[0] - pp1[ii]) > k :             # while (l != () and |l(0) - pos(pp1)| > k)
                                        l.remove(l[0])                                  # delete(l[0])
                                for ps in l:                                            # for each ps in l
                                        answer.append([ docID(p1[i]), pp1[ii], ps ])    # add answer(docID(p1), pos(pp1), ps)
                                ii+=1                                                   # pp1 <- next(pp1)
                        i+=1                                                            # p1 <- next(p1)
                        j+=1                                                            # p2 <- next(p2)
                elif docID(p1[i]) < docID(p2[j]):                                       # else if (docID(p1) < docID(p2))
                        i+=1                                                            # p1 <- next(p1)                                                        
                else:
                        j+=1                                                            # p2 <- next(p2)
        return answer

In [28]:
# multi word query 
# this sorted query question by frequency
def query_multi():
  sorted_dicts = []
  query = input("enter words for checking: ")
  time_start = datetime.datetime.now()
  query_split = query.split(" ")
  for q in range(len(query_split)):
    query_split[q] = convert_en_characters(query_split[q])
  k = 1
  if len(query_split) < 2:
    print("not enough word:(")
    return 
  for i in range(len(query_split)):
    normal_query = lstemmer.stem(query_split[i])
    print("Normal word to search {}".format(normal_query))
    all , diction =  sort_dict(dictionary[normal_query])
    print(all)
    print(diction)
    sorted_dicts.append((all , diction))
  print(sorted_dicts)
  sorted_dicts = sorted(sorted_dicts, key = lambda x: x[1])
  for x in range(len(sorted_dicts)-1):
    if(sorted_dicts[x][0] == 0):
      sorted_dicts.pop(x)
  print(len(sorted_dicts))
  print("sorted_dicts: ", sorted_dicts)
  if len(sorted_dicts) > 2:
    sorted_dicts1 = position_intersect(sorted_dicts[0][1] ,sorted_dicts[1][1] , k)
    sorted_dicts1 = sorted(sorted_dicts1, key = lambda x: x[1])
    sorted_dicts2 = position_intersect(sorted_dicts[1][1] ,sorted_dicts[2][1] , k)
    sorted_dicts2 = sorted(sorted_dicts2, key = lambda x: x[1])
    print("sorted_dicts1: ", sorted_dicts1)
    print("sorted_dicts2: ",sorted_dicts2)

  elif (len(sorted_dicts) == 2):
    sorted_dicts = position_intersect(sorted_dicts[0][1] ,sorted_dicts[1][1] , k)
    sorted_dicts = sorted(sorted_dicts, key = lambda x: x[1])
    print(sorted_dicts)
    res = collections.defaultdict(list)
    for x in range(len(sorted_dicts)):
      res[sorted_dicts[x][0]].append([sorted_dicts[x][1] , sorted_dicts[x][2]])
    res = sorted(res, key=lambda k: len(res[k]), reverse=True)
  time_finish = datetime.datetime.now()
  print("{} results in {} ms".format(len(res), ((time_finish - time_start).total_seconds())*1000))
  print("id -> title\n")
  for i in range(len(res)):
    print("{} -> {}".format(res[i] , title[res[i]]))


In [ ]:
query_multi()

In [38]:
def zipfs_law():
  print(sorted(length_dict_non_posting, key=lambda x: x,reverse=True)[:20])


In [39]:
zipfs_law()

['💬', 'ﻭ', 'ﻫﻤﭽﻨﯿﻦ', 'ﻫﯿﺄﺕ', 'ﻧﻮﺑﻪ', 'ﻧﻮﭘﺎی', 'ﻧﻤﺎد', 'ﻧﻈﺎم', 'ﻧﺎﺭﻭﺍ', 'ﻧﺎﮐﺎم', 'ﻣﻮﻗﻌﯿﺖ', 'ﻣﻮﺍﺭﺩ', 'ﻣﻬﻤﺘﺮﯾﻦ', 'ﻣﻬﻢ', 'ﻣﻨﻄﻘﻪ\u200cای', 'ﻣﻨﺪﯼ', 'ﻣﻨﺎﻃﻖ', 'ﻣﻠﺖ', 'ﻣﻘﺮﺭﺍﺕ', 'ﻣﺮﺣﻠﻪ']


In [12]:
def heap_law():
  count = 1000
  normalizer = Normalizer()
  dictionary_heap = collections.defaultdict(list)
  for i in range(count):
    tokenizes = []
    tokenizes = word_tokenize(content[i])
    tokenizes = stem_stop(tokenizes)
    tokenizes_count = dict(collections.Counter(tokenizes))
    for key , value in tokenizes_count.items():
        if key not in stopwords:
          index = []
          pos_index = position_find(key , tokenizes)
          index.append(i)
          index.append(value)
          index.append(pos_index)
          dictionary_heap[key].append(index)
  sum = 0
  for key , value in dictionary_heap.items():
    for x in value:
      sum += x[1]
  
  print("sum for tokens for ",count," document is: ",sum," and length of vocabulary(M) is : ", len(list(dictionary_heap.keys())))

In [17]:
heap_law()

sum for tokens for  1000  document is:  142881  and length of vocabulary(M) is :  9195


## PHASE II

In [ ]:
# Phase II
example = copy.deepcopy(dictionary[lstemmer.stem('یحیی')])
N = len(content)
for x in example:
    n = len(example)
    tfidf = (1+math.log((x[1]),10)) * math.log((N/n),10)
    x.append(tfidf)
print(dictionary[lstemmer.stem('یحیی')])
print(example)

[[1, 1, [126]], [47, 2, [160, 163]], [49, 1, [374]], [84, 2, [60, 63]], [128, 1, [50]], [132, 2, [55, 58]], [177, 2, [482, 485]], [223, 1, [6]], [226, 2, [19, 22]], [227, 1, [248]], [231, 1, [168]], [237, 1, [4]], [251, 2, [6, 9]], [268, 2, [4, 7]], [485, 2, [23, 100]], [532, 2, [246, 249]], [601, 1, [50]], [647, 2, [6, 9]], [652, 1, [15]], [653, 1, [183]], [654, 1, [163]], [656, 1, [89]], [659, 1, [104]], [665, 2, [28, 31]], [733, 1, [90]], [736, 1, [6]], [768, 1, [75]], [786, 2, [6, 9]], [791, 2, [83, 86]], [809, 2, [314, 317]], [814, 2, [43, 46]], [816, 1, [139]], [828, 1, [134]], [863, 2, [199, 202]], [864, 2, [78, 81]], [880, 1, [142]], [894, 2, [62, 240]], [928, 1, [14]], [932, 1, [57]], [964, 2, [337, 340]], [986, 1, [261]], [995, 2, [15, 18]], [1013, 2, [16, 83]], [1018, 1, [48]], [1043, 1, [361]], [1087, 1, [11]], [1105, 1, [6]], [1124, 3, [430, 815, 818]], [1133, 2, [16, 19]], [1157, 1, [30]], [1165, 1, [11]], [1177, 1, [35]], [1183, 1, [541]], [1191, 1, [137]], [1211, 1, [12

In [ ]:
# Make the tfidf for all of the words
def calculate_tfidf(term_dict):
  N = len(content)
  doc_norm = [0] * N
  Champion_list = {}
  print("The contents size : " , N)
  for key , value in term_dict.items():
      value = term_dict[key]
      df = len(value)
      # print(term_dict[key] , key)
      if(df != 0):
        # the number of docs that we have value in 
        idf =  math.log((N/df),10)
        for x in value:
          # calcualte the tfidf for each docs
          tfidf = (1+math.log((x[1]),10)) * idf 
          doc_norm[x[0]] += tfidf**2 
          x.append(tfidf)
        value = sorted(value, key=operator.itemgetter(3), reverse=True)
        new_value = [idf, value]
        term_dict[key] = new_value
  return term_dict , doc_norm

In [ ]:
# Make the documents size
def Doc_size(term_dict):
  N = len(content)
  doc_size = [0] * N
  for key,value in term_dict.items():
      for i in range(len(value)):
            doc_size[value[i][0]] = doc_size[value[i][0]] + value[i][1]
  return doc_size

In [ ]:
# Nomilaze the vectors
def Vector_Normalize(term_dict , doc_norm):
    print("In Vector Normiliziton")
    print(term_dict['یح'])
    for key , value in term_dict.items():
      for i in range(len(value)):
        value[i][3] = value[i][3]/math.sqrt(doc_norm[value[i][0]])
    print(term_dict['یح'])
    return term_dict

In [ ]:
# The Token Normalizetion
def Normalize_tokens(tokens):
  for i in range(len(tokens)):
    tokens[i] = lstemmer.stem(tokens[i])
  return tokens

In [ ]:
# query vector Normalizition
def query_vector_normalize(query_dict):
  sigma = 0
  for key, value in query_dict.items():
    sigma = sigma + value**2
  for key, value in query_dict.items():
    query_dict[key] = value / math.sqrt(sigma)
  return query_dict

In [ ]:
# Vector of the query 
def vectorize_query(query):
    query_token =  word_tokenize(query)
    normal_query_token = Normalize_tokens(query_token)
    # print(normal_query_token)
    term_freq_dict = dict(collections.Counter(normal_query_token))
    query_terms = {}
    for  term,freq in term_freq_dict.items():
        if term in dictionary.keys() and term not in stopwords:
            query_terms[term] = int(1+math.log(freq,10))
    # query_terms = query_vector_normalize(query_terms)
    return query_terms

In [ ]:
# Calculate the Similiraty of query and docs
def CosineScore(query_dict , terms, doc_norm, K):
  N = len(content)
  scores_dict = {key:0 for key in range(N)}
  # scores_dict = [0 for i in range(0, N)]
  Final_score = []
  founded_doc = 0
  for key , value in query_dict.items():
    term = terms[key]
    idf_term = term[0]
    w_q = query_dict[key] * idf_term
    postings = term[1]
    # print(len(postings))
    champion_list = postings[:50]
    champion_list = postings
    for x in range(len(champion_list)):
      w_d =  champion_list[x][3]
      scores_dict[champion_list[x][0]] += w_q * w_d
      founded_doc += 1
  # print({k: v for k, v in sorted(scores_dict.items(), key=lambda item: item[1], reverse=True)})

  if founded_doc < K:
    print("Doc it`s not enough in Chmpion lists")
    for key , value in query_dict.items():
      if founded_doc < K:
        term = terms[key]
        idf_term = term[0]
        w_q = query_dict[key] * idf_term
        postings = term[1]
        above_doc = postings[:25]
        for x in range(len(above_doc)):
          w_d =  above_doc[x][3]
          scores_dict[above_doc[x][0]] += w_q * w_d
          founded_doc += 1
      
  for i in range(len(scores_dict)):
    if doc_norm[i] == 0 or scores_dict[i] == 0:
      continue
    scores_dict[i] = scores_dict[i] / math.sqrt(doc_norm[i])
    Final_score.append([i , scores_dict[i]])
  for i in range(len(Final_score)):
    for j in range(i+1, len(Final_score)):
      if Final_score[j][1] > Final_score[i][1]:
        Final_score[i], Final_score[j] = Final_score[j], Final_score[i]
  return Final_score

In [ ]:
# Show the results 
def Show_res(Scores , K, normal_query):
  print("id -> title\n")
  print("The Five Top Related Docs\n")
  for i in range(K):
    print("{} -> {}".format(Scores[i][0] , title[Scores[i][0]]))
  find_secntences(Scores[0][0] , normal_query)
  print(title[Scores[0][0]])
  print(content[Scores[0][0]])

In [ ]:
def find_secntences(doc_id , query_token):
  sentences = sent_tokenize(df.loc[doc_id].content)
  for sen in sentences:
    if(query_token[0] in sen):
        print(sen)

In [ ]:
dictionarys = copy.deepcopy(dictionary)
dictionarys , doc_norm = calculate_tfidf(dictionarys)

In [ ]:
query = input("enter a word for checking: ")
time_start = datetime.datetime.now()
normal_query = lstemmer.stem(query)
print("Normal word to search {}".format(normal_query))
query_dict = vectorize_query(query)
K = 5
Scores = CosineScore(query_dict , dictionarys , doc_norm, K)
time_finish = datetime.datetime.now()
print("{} results in {} ms".format(len(Scores), ((time_finish - time_start).total_seconds())*1000))
Show_res(Scores, K, normal_query)

In [ ]:
query_one_word()  

In [ ]:
query_multi()

##This is the Preferential part of Phase II

In [ ]:
import pickle
import multiprocessing
from gensim.models import Word2Vec
import time
from numpy.linalg import norm

In [ ]:
cores = multiprocessing.cpu_count()
print("# cores " , cores)

# cores  2


In [ ]:
handle = open('')
traning_data = pickle.load(handle)

In [ ]:
traning_data = content
docs_num = len(traning_data)
tokens_num = sum([len(x) for x in traning_data])
print(docs_num)
print(tokens_num)

7562
13078523


In [ ]:

w2v_model = Word2Vec(min_count = 1,
                     window = 5,
                     vector_size = 300,
                     alpha = 0.03,
                     worker = cores-1)
w2v_model.build_vocab(traning_data)
w2v_model_vocab_size = len(w2v_model.wv)


TypeError: ignored